# Not Standardizated Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("Preprocessed_before_standardization_and_normalization.csv")
df.head()

,price,m2_price,square,rooms,district,micro_district,building_type,floor,floors,condition,building_age,floor_to_floors,is_good_floor
0,110880,720,154,4,1,0,2,12,14,2,2,0.857143,0
1,57000,740,77,2,1,0,2,5,12,2,2,0.416667,1
2,57750,750,77,2,1,0,1,5,12,2,2,0.416667,1
3,57000,740,77,2,1,0,2,5,12,2,2,0.416667,1
4,55000,1279,43,1,2,3,2,1,12,6,12,0.083333,0


### troubleshooting unnoticed problems

In [3]:

df['rooms'] = df['rooms'].replace({
    '6 и более': 6,
    'свободная планировка': 7
}).astype(int)

In [4]:
df.drop(["floor", "floors"], axis=1, inplace=True)# no need columns

## Dividing into train and test data

In [5]:
X = df.drop(columns=['price'])  # goal variable
y = df['price']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Trying a models

### DecisionTreeRegressor

In [7]:
from sklearn.tree import DecisionTreeRegressor

dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("Decision Tree Regressor:")
print("R² Score:", r2_score(y_test, y_pred_dt))
print("MSE:", mean_squared_error(y_test, y_pred_dt))

Decision Tree Regressor:
R² Score: 0.9956338784789327
MSE: 5838587.156765535


### GradientBoostingRegressor

In [8]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)

print("Gradient Boosting Regressor:")
print("R² Score:", r2_score(y_test, y_pred_gb))
print("MSE:", mean_squared_error(y_test, y_pred_gb))

Gradient Boosting Regressor:
R² Score: 0.9974636423265404
MSE: 3391739.1592905014


### cross-validation of gb_model

In [9]:
from sklearn.model_selection import cross_val_score
import numpy as np

# 5-fold cross-validation
cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='r2')

print("Cross-validation R² Scores:", cv_scores)
print("Mean Cross-validation R² Score:", np.mean(cv_scores))


Cross-validation R² Scores: [0.99675298 0.9968115  0.99716674 0.99724117 0.99760398]
Mean Cross-validation R² Score: 0.9971152739671416


In [10]:
# deleting m2_price from tainig data
X_train_new = X_train.drop(columns=['m2_price'])
X_test_new = X_test.drop(columns=['m2_price'])

from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_new, y_train)

GradientBoostingRegressor(random_state=42)

In [11]:
from scipy.stats import ttest_ind

# Пример данных
sample1 = X_train['square']  # Выборка 1 (например, площадь квартир в обучающей выборке)
sample2 = X_test['square']   # Выборка 2 (например, площадь квартир в тестовой выборке)

# t-тест
t_stat, p_value = ttest_ind(sample1, sample2)

print("t-Statistic:", t_stat)
print("p-Value:", p_value)

if p_value < 0.05:
    print("Гипотеза о равенстве средних отвергается.")
else:
    print("Нет оснований отвергнуть гипотезу о равенстве средних.")

t-Statistic: -0.0428688750179904
p-Value: 0.9658069458939689
Нет оснований отвергнуть гипотезу о равенстве средних.


In [12]:
from scipy.stats import chi2_contingency
import numpy as np

# Пример таблицы сопряженности
contingency_table = pd.crosstab(df['district'], df['building_type'])

# Хи-квадрат тест
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print("Chi-Square Statistic:", chi2_stat)
print("p-Value:", p_value)
print("Degrees of Freedom:", dof)
print("Expected Frequencies:\n", expected)

if p_value < 0.05:
    print("Гипотеза о независимости отвергается.")
else:
    print("Нет оснований отвергнуть гипотезу о независимости.")

Chi-Square Statistic: 615.2550660951492
p-Value: 1.1937236949656734e-129
Degrees of Freedom: 6
Expected Frequencies:
 [[ 224.48468543  491.68201573  328.83329884]
 [1078.60057947 2362.42622103 1579.9731995 ]
 [ 422.11713576  924.55039321  618.33247103]
 [ 350.79759934  768.34137003  513.86103063]]
Гипотеза о независимости отвергается.


In [108]:
feature_columns = [
    'square', 'rooms', 'district', 'micro_district',
    'building_type', 'condition', 'building_age',
    'floor_to_floors', 'is_good_floor'
]

micro_district_options = {
    4: ['Золотой квадрат', 'Асанбай м-н', 'Бишкек-Парк', 'ЦУМ', 'Орозбекова - Жибек-Жолу',
        'Горького - Панфилова', 'Московская - Белинка', 'Московская - Уметалиева',
        'Парк Панфилова/Спартак', 'Филармония'],
    3: ['Академия Наук', 'КНУ', 'АУЦА', 'Дворец спорта', 'Мед. академия', 'Ак Кеме',
        'Молодая Гвардия', 'Восток-5 м-н', 'Юг-2 м-н', 'Кок-Жар м-н', 'Центральная мечеть',
        'Парк Ататюрк', 'Ген прокуратура'],
    2: ['4 м-н', '5 м-н', '6 м-н', '7 м-н', '8 м-н', '9 м-н', '10 м-н', '11 м-н', '12 м-н',
        'Джал 15 м-н', 'Джал-23 м-н', 'Джал-29 м-н', 'Джал-30 м-н', 'Средний Джал м-н',
        'Верхний Джал м-н', 'Улан м-н', 'Тунгуч м-н', 'Аламедин-1 м-н', 'Политех',
        'КГУСТА', 'БГУ', 'Азия Молл', 'Советская - Скрябина', 'Карпинка', 'Матросова'],
    1: ['ЖД вокзал', 'Военторг', 'АЮ Grand', 'Ипподром', 'Площадь Победы', 'Моссовет',
        'Старый аэропорт', 'Гоин', 'Душанбинка', 'Восточный автовокзал', 'Юбилейка',
        'Космос', 'Кара-Жыгач ж/м', 'Джальская больница', 'Нижний Токольдош',
        'Кудайберген', 'Пишпек ж/м', 'ТЭЦ', 'VEFA', 'Щербакова ж/м', 'Ак Эмир рынок',
        'Госрегистр', 'Церковь', 'Чуй - Алматинка', 'Западный автовокзал', 'Таатан',
        'Гагарина', 'Учкун м-н', 'Городок энергетиков', 'Аламединский рынок',
        'Жилгородок Совмина ж/м', 'Улан-2 м-н'],
    0: ['Другие/самые дешёвые районы']
}

district_options = {
    3: ['Первомайский район'],
    2: ['Ленинский район'],
    1: ['Октябрьский район'],
    0: ['Свердловский район']
}

building_type_map = {
    2: 'кирпичный',
    1: 'монолитный',
    0: 'панельный'
}

condition_map = {
    0: 'не достроено',
    1: 'черновая отделка',
    2: 'под самоотделку',
    3: 'требует ремонта',
    4: 'среднее',
    5: 'евроремонт',
    6: 'хорошее',
    7: 'свободная планировка'
}

#input function
def get_user_input():
    data = {}

    print("\n Select *micro_district* group:")
    for k, v in micro_district_options.items():
        print(f"\n{k}:")
        for name in v:
            print(f"   - {name}")
    data['micro_district'] = int(input("\nEnter micro_district group (0-4): "))

    print("\n Select *district* group:")
    for k, v in district_options.items():
        print(f"{k}: {', '.join(v)}")
    data['district'] = int(input("\nEnter district group (0-3): "))

    data['square'] = float(input(" Enter apartment area in m²: "))
    data['rooms'] = int(input(" Enter number of rooms: "))

    print("\n Select building type:")
    for k, v in building_type_map.items():
        print(f"{k}: {v}")
    data['building_type'] = int(input("Enter building type (0-2): "))

    print("\n Select apartment condition:")
    for k, v in condition_map.items():
        print(f"{k}: {v}")
    data['condition'] = int(input("Enter condition (0-7): "))

    data['building_age'] = int(input(" Enter building age (years): "))
    data['floor_to_floors'] = float(input(" Enter floor/floors ratio (e.g. 4/9): "))
    
    print("\n Is it a good floor? (1 = yes, 0 = no)")
    data['is_good_floor'] = int(input("Enter 1 or 0: "))

    return pd.DataFrame([data])
user_df = get_user_input()
user_df = user_df[feature_columns]

# prediction
predicted_price = gb_model.predict(user_df)[0]
square = user_df.iloc[0]['square']
predicted_m2_price = predicted_price / square

# result
print(f"\n🔮 Predicted apartment price: {int(predicted_price):,} $")
print(f"📐 Price per m²: {int(predicted_m2_price):,} $")


🗺️ Select *micro_district* group:

4:
   - Золотой квадрат
   - Асанбай м-н
   - Бишкек-Парк
   - ЦУМ
   - Орозбекова - Жибек-Жолу
   - Горького - Панфилова
   - Московская - Белинка
   - Московская - Уметалиева
   - Парк Панфилова/Спартак
   - Филармония

3:
   - Академия Наук
   - КНУ
   - АУЦА
   - Дворец спорта
   - Мед. академия
   - Ак Кеме
   - Молодая Гвардия
   - Восток-5 м-н
   - Юг-2 м-н
   - Кок-Жар м-н
   - Центральная мечеть
   - Парк Ататюрк
   - Ген прокуратура

2:
   - 4 м-н
   - 5 м-н
   - 6 м-н
   - 7 м-н
   - 8 м-н
   - 9 м-н
   - 10 м-н
   - 11 м-н
   - 12 м-н
   - Джал 15 м-н
   - Джал-23 м-н
   - Джал-29 м-н
   - Джал-30 м-н
   - Средний Джал м-н
   - Верхний Джал м-н
   - Улан м-н
   - Тунгуч м-н
   - Аламедин-1 м-н
   - Политех
   - КГУСТА
   - БГУ
   - Азия Молл
   - Советская - Скрябина
   - Карпинка
   - Матросова

1:
   - ЖД вокзал
   - Военторг
   - АЮ Grand
   - Ипподром
   - Площадь Победы
   - Моссовет
   - Старый аэропорт
   - Гоин
   - Душанбинка
   


Enter micro_district group (0-4):  1



🏙️ Select *district* group:
3: Первомайский район
2: Ленинский район
1: Октябрьский район
0: Свердловский район



Enter district group (0-3):  3
📐 Enter apartment area in m²:  76
🛏️ Enter number of rooms:  3



🏗️ Select building type:
2: кирпичный
1: монолитный
0: панельный


...


🎨 Select apartment condition:
0: не достроено
1: черновая отделка
2: под самоотделку
3: требует ремонта
4: среднее
5: евроремонт
6: хорошее
7: свободная планировка


Enter condition (0-7):  4
🏢 Enter building age (years):  4
📊 Enter floor/floors ratio (e.g. 4/9):  0.9



⬆️ Is it a good floor? (1 = yes, 0 = no)


Enter 1 or 0:  1



🔮 Predicted apartment price: 71,754 $
📐 Price per m²: 944 $
